# Generate Fitting DataFrame from HITRAN call using HAPI calls

In [6]:
#Import Statements
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.set_context("poster")
sys.path.append(r'C:\Users\ema3\Documents\Python Scripts\HAPI')#Add hapi.py folder location to system path
from hapi import *

pd.set_option("display.max_rows", 101)

In [7]:
#Set Working File location
spec_path = r'C:\Users\ema3\Documents\Python Scripts\HAPI' # Location of the Summary Data File
os.chdir(spec_path)

In [3]:
# This call prints the dictionary ISO_ID that provides information on the Global ideas for isotopologues in HITRAN with their molecule number, molecule specific isotope number, 
# abundance, mass, and molecule name
print_iso_id() 

The dictionary "ISO_ID" contains information on "global" IDs of isotopologues in HITRAN

   id            M    I                    iso_name       abundance       mass        mol_name
    1     :      1    1                     H2(16O)    0.9973170000  18.010565             H2O
    2     :      1    2                     H2(18O)    0.0019998300  20.014811             H2O
    3     :      1    3                     H2(17O)    0.0003720000  19.014780             H2O
    4     :      1    4                     HD(16O)    0.0003106900  19.016740             H2O
    5     :      1    5                     HD(18O)    0.0000006230  21.020985             H2O
    6     :      1    6                     HD(17O)    0.0000001160  20.020956             H2O
    7     :      2    1                 (12C)(16O)2    0.9842040000  43.989830             CO2
    8     :      2    2                 (13C)(16O)2    0.0110570000  44.993185             CO2
    9     :      2    3             (16O)(12C)(18O)    0

In [9]:
#Set-Up Variables for lineList
tablename = 'Singlet Delta'
global_isotopes = [36, 37, 38]
wave_min = 7700 #7903.5#cm-1
wave_max = 8050 #7904.5 #cm-1
intensity_cutoff = 1e-30


In [10]:
def HITRANlinelist_to_csv(isotopes, minimum_wavenumber, maximum_wavenumber, tablename = 'tmp', filename = tablename, temperature = 296): 
    #Retrieves linelist from HITRAN will generate two dataframes, one is the list of everything in HITRAN the other will be an initial guess file for your fitting
    db_begin('data')
    fetch_by_ids(tablename, global_isotopes, minimum_wavenumber, maximum_wavenumber, ParameterGroups=['all']) # pulls down all HITRAN data for the data
    cond = ('AND',('between','nu',minimum_wavenumber,maximum_wavenumber),('>=','sw',intensity_cutoff))
    select(tablename,Conditions=cond, DestinationTableName='tmp')
    
    
    #Generates Pandas dataframe using linelist
    ## First Generates a table that contains everything that is in HITRAN for that molecule in that waverange    
    linelist = pd.DataFrame()
    standard = ['trans_id','molec_id','local_iso_id','nu','sw','a','elower','gp','gpp','elower', 'global_upper_quanta', 'global_lower_quanta', 'local_upper_quanta', 'local_lower_quanta']
    for par_name in standard: # adds standard items to the linelist
        linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
    for par_name in PARLIST_HT_AIR:
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
    for par_name in PARLIST_HT_SELF:
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass 
    
    for par_name in PARLIST_VOIGT_AIR: # adds VP air parameters
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
        
    for par_name in PARLIST_VOIGT_SELF: # adds VP self parameters
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
        
    for par_name in PARLIST_VOIGT_H2:
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
    for par_name in PARLIST_VOIGT_CO2:
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
    for par_name in PARLIST_VOIGT_HE:
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
    for par_name in PARLIST_VOIGT_H2O:
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
    for par_name in ['y_air', 'y_self']:
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
    for par_name in ['SD_air', 'SD_self', 'SD_H2', 'SD_CO2', 'SD_He', 'SD_H2O']: # Addition of Speed Dependent Parameters
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass            
    for par_name in ['beta_g_air', 'beta_g_self', 'beta_g_H2', 'beta_g_CO2', 'beta_g_He', 'beta_g_H2O']: # Addition of Speed Dependent Parameters
        try:
            linelist[par_name] = LOCAL_TABLE_CACHE['tmp']['data'][par_name]
        except:
            pass
    linelist.to_csv(filename + '_HITRAN.csv', index = False)
    
    # Next segment looks at at all possible data and makes an initial guess fill prioritizing HTP over non HTP parameters, but will mix them. 
    # It will also calculate aw based on theory and set values where there is an air value, but no self value equal to the air value.  COmment line printed at end will detail these
    #  Will also use GP values if available (and NGP from HTP are not) wil make a note in comment if this happens
    avail_species = []
    linelist_select = linelist[['trans_id','molec_id','local_iso_id','nu','sw','a','elower','gp','gpp','elower', 'global_upper_quanta', 'global_lower_quanta', 'local_upper_quanta', 'local_lower_quanta']]   
    for param in list(linelist):
        if ('gamma_' in param) and ('HT' not in param):
            avail_species.append(param[6:])
            
    #define reference temperature and pressure
    Tref = 296. # K
    
    # define actual temperature and pressure
    T = temperature # K
    TRanges = [(0,100),(100,200),(200,400),(400,float('inf'))]
    Trefs = [50.,150.,296.,700.]
    for TRange,TrefHT in zip(TRanges,Trefs):
        if T >= TRange[0] and T < TRange[1]:
            break
    comment = ''       
    for species in avail_species:
        #Gamma0
        try:
            linelist_select.loc[linelist_select.index,'gamma0_%s'%(species)] = linelist['gamma_HT_0_%s_%d'%(species,TrefHT)].values
        except:
            linelist_select.loc[linelist_select.index, 'gamma0_%s'%(species)] = linelist['gamma_%s'%species].values
        #Temperature Dependence Gamma0
        try: 
            linelist_select.loc[linelist_select.index,'n_gamma0_%s'%(species)] = linelist['n_HT_%s_%d'%(species,TrefHT)].values
        except:
            try:
                linelist_select.loc[linelist_select.index,'n_gamma0_%s'%(species)] = linelist['n_%s'%species].values
            except:
                linelist_select.loc[linelist_select.index,'n_gamma0_%s'%(species)] = linelist['n_air'].values
        if (linelist_select['n_gamma0_%s'%(species)] == 0).all():
            linelist_select.loc[linelist_select.index,'n_gamma0_%s'%(species)] = linelist['n_air'].values
            comment += 'set n_gamma0_%s'%(species) + ' to n_gamma0_air'

        #Delta0
        try:
            linelist_select.loc[linelist_select.index,'delta0_%s'%(species)] = linelist['delta_HT_0_%s_%d'%(species,TrefHT)].values
        except:
            try:
                linelist_select.loc[linelist_select.index,'delta0_%s'%(species)] = linelist['delta_%s'%species].values
            except:
                linelist_select.loc[linelist_select.index,'delta0_%s'%(species)] = linelist['delta_air']
                comment += 'set delta0_%s'%(species) + ' to delta0_air'
        #Temperature Dependence of Delta0
        try:
            linelist_select.loc[linelist_select.index,'n_delta0_%s'%(species)] = linelist['deltap_HT_%s_%d'%(species,TrefHT)].values
        except:
            try:
                linelist_select.loc[linelist_select.index,'n_delta0_%s'%(species)] = linelist['deltap_%s'%species].values
            except:
                try:
                    linelist_select.loc[linelist_select.index,'n_delta0_%s'%(species)] = linelist['deltap_air'].values
                    comment += 'set n_delta0_%s'%(species) + ' to n_delta0_air'
                except:
                    linelist_select.loc[linelist_select.index,'n_delta0_%s'%(species)] = 0
        
        #Speed Dependent Broadening
        try:
            linelist_select.loc[linelist_select.index,'SD_gamma_%s'%(species)] = linelist['gamma_HT_2_%s_%d'%(species,TrefHT)].values
        except:
            try:
                linelist_select.loc[linelist_select.index,'SD_gamma_%s'%(species)] =linelist['SD_%s'%species].values
            except:
                try: 
                    linelist_select.loc[linelist_select.index,'SD_gamma_%s'%(species)] =linelist['SD_air'].values
                except:
                    #linelist_select.loc[linelist_select.index,'SD_gamma_%s'%(species)] = 0
                    for i in range(0, len(linelist)):
                        if species == 'air':
                            m_p = 28.97
                        elif species == 'H2':
                            m_p = 2.01588
                        elif species == 'CO2':
                            m_p = 43.98983
                        elif species == 'HE':
                            m_p = 4.002602
                        elif species == 'H2O':
                            m_p = 18.010565
                        else:
                            m_p = np.float(molecularMass(linelist.loc[i]['molec_id'],linelist.loc[i]['local_iso_id']))
                        m_a = np.float(molecularMass(linelist.loc[i]['molec_id'],linelist.loc[i]['local_iso_id']))
                        aw = (1-linelist_select.loc[i,'n_gamma0_%s'%(species)])*(2/3) * ((m_p / m_a) / (1 + (m_p / m_a) ))
                        linelist_select.loc[i,'SD_gamma_%s'%(species)] = aw          

        #Temperature Dependence of SD Broadening
        linelist_select.loc[linelist_select.index,'n_gamma2_%s'%(species)] = linelist_select['n_gamma0_%s'%(species)].values
        
        #Speed Dependent Shift
        try:
            linelist_select.loc[linelist_select.index,'SD_delta_%s'%(species)] = linelist['delta_HT_2_%s_%d'%(species,TrefHT)].values
        except:
            linelist_select.loc[linelist_select.index,'SD_delta_%s'%(species)] = 0
        # Temperature Dependence of SD Shift
        linelist_select.loc[linelist_select.index,'n_delta2_%s'%(species)] = linelist_select['n_delta0_%s'%(species)].values
        
        #nuVC
        try:
            linelist_select.loc[linelist_select.index,'nuVC_%s'%(species)] = linelist['nu_HT_%s'%species].values
        except:
            try:
                linelist_select.loc[linelist_select.index,'nuVC_%s'%(species)] = linelist['beta_g_%s'%species].values
                comment += ' nuVC_' + species + ' = beta_g_' + species
            except:
                linelist_select.loc[linelist_select.index,'nuVC_%s'%(species)] = 0
        #nuVC temperature dependence
        try:
            linelist_select.loc[linelist_select.index,'n_nuVC_%s'%(species)] = linelist['kappa_HT_%s'%species].values
        except:
            linelist_select.loc[linelist_select.index,'n_nuVC_%s'%(species)] = 1
            
        #eta
        try:
            linelist_select.loc[linelist_select.index,'eta_%s'%(species)] = linelist['eta_HT_%s'%species].values
        except:
            linelist_select.loc[linelist_select.index,'eta_%s'%(species)] = 0  
            
        # Linemixing
        try:
            linelist_select.loc[linelist_select.index,'y_%s'%(species)] = linelist['y_%s'%species].values
        except:
            linelist_select.loc[linelist_select.index,'y_%s'%(species)] = 0
  
    print (comment)    
    linelist_select.to_csv(filename + '_initguess.csv', index = False)
               
    return linelist_select    

    


linelist_select = (HITRANlinelist_to_csv(global_isotopes, wave_min, wave_max, tablename = tablename))


Using data

CO2
                     Lines parsed: 300
Oxygen
                     Lines parsed: 488
TestTable
                     Lines parsed: 33

Data is fetched from http://hitran.org

BEGIN DOWNLOAD: Singlet Delta
  65536 bytes written to data/Singlet Delta.data
  65536 bytes written to data/Singlet Delta.data
  65536 bytes written to data/Singlet Delta.data
  65536 bytes written to data/Singlet Delta.data
  65536 bytes written to data/Singlet Delta.data
  65536 bytes written to data/Singlet Delta.data
Header written to data/Singlet Delta.header
END DOWNLOAD
                     Lines parsed: 941
PROCESSED
 nuVC_air = beta_g_airset n_gamma0_self to n_gamma0_air nuVC_self = beta_g_self
